<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Introduction to Web Services and APIs

_Authors: Dave Yerrington (SF)_

---
![](assets/opening.png)

### Learning Objectives
*After this lesson, you will be able to:*
- Describe APIs and how to make calls and consume API data.
- Access public APIs and get information back.
- Read and write data in JSON format.
- Identify HTTP Verbs & their uses.
- Use the `requests` library.

## Introduction to APIs

---

In previous classes we learned about building scripts that scrape content from web-sites.  In this lesson we will be diving into the world of APIs and taking a tour of one of the most accessible sources of data on the internet.

You will learn:
- **What is meant by an "API"**
- **Common use cases**
- **How to read API documentation**
- **General development workflow with APIs**


## But first, a question:

---
Suppose you're working on the data team for a startup interested in providing weather visualizations to consumers via online and mobile. Your boss needs you to build a dashboard that visualizes historical weather data hour-by-hour for a certain location on a certain day for your product. How would you go about sourcing that data?

### Scenario 1 (Best Case)
---
- Your company has a SQL database or other organized database already built with the data you'll need and you have access to it. 

### Scenario 2 ("Worst" Case&ast;)
---
- You know of a website or collection of sites with the data you'll need and you build a scraper to pull the data down. Depending on the complexity of the site(s) this could take some time and does not scale well. You're also vulnerable to site design changes, so maintaining over long periods of time can become cumbersome. 

*&ast; There is a worse case - the data you're after does not exist in any structured form at all!*

### Scenario 3 (Goldilocks Case)
---
- There's an API that contains the data you're after. You can make requests to this API and pull down the data you'll need. 

## What is an API?

---

An API (Application Programming Interface) is a set of routines, protocols, and tools for building software applications. It specifies how software components should interact.

APIs are a way developers abstract functionality to data, devices, and other resources they provide. 

Some examples include:

- Connectivity to a variety of databases
- Python modules that can turn LED lights on and off
- Application that runs on native Windows, OSX, or Linux
- Libraries that post content on Twitter, Facebook, Yelp, or LinkedIn
- Web services for accessing currency or stock prices

More abstract examples:
- Adding your own functions to Numpy itself
- Extending Python with C code
- Testing Frameworks
- A waiter at a restaurant
- Your TV Remote

In the context of data science, APIs are a very common method to interact with data hosted by third parties and most commonly provided by **Web Service APIs**.

### API Spotlight:  Darksky.net

Darksky.net provides an API for interacting with their weather data.  At a glance:

- View historical weather data for a latitude/longitude pair
    - Can be broken down granularly to the
        - Day
        - Hour
        - Minute
- View an hour-by-hour forecast of the weather for the next 48 hours.
- View a day-by-day forecast for the next 7 days. 

### Example

Plugging the address below into our browser lets us hit the Darksky API (you must sign up for a key and replace "YouKeyHere" with your key). This gives us the weather for NYC on August 8, 2017 by day and hour. 

https://api.darksky.net/forecast/YourKeyHere/40.7808,-73.9772,1502150400?exclude=minutely,currently

### API Spotlight:  The New York Times

The New York Times provides an API that allows developers to access:

- Articles - NYT articles by month dating back to 1851.
    - Arguments you can send to this API include:
        - A search query (API will return any matches result from your query and content in the article body, headline and byline.)
        - Article date range (Start date, end date) - Restricts results to those articles with matching start and end dates, respectively. 
- Books - Information about book reviews and NYT bestsellers.
- Most Popular NYT articles based on shares, views and emails. 
- Movie Reviews - NYT Critics' Picks

### API Spotlight:  Spotify

The Spotify API allows developers to access their database of information on:

- Albums
    - Name of Album
    - Album Type
    - Artists
    - Available Markets
    - Copyrights
    - Genres
    - Album art images
    - Popularity
- Playlists
    - Whether a playlist can be modified by other users or not
    - Description of playlist
    - Followers of playlist
    - Public/Private Status of playlist
    - Tracks on Playlist
- Artists, songs etc.!

## Web APIs

---

![](assets/notify.png)

The prevelance of web APIs have increased 10x with the rise of Javascript and advent of web programming techniques allowing for the transmission of small pieces of data without having to refresh the entire page.

With the growth of highly interactive websites, provided by AJAX programming techniques in Javascript, many languages have started co-opting standards to communicate data to and from web servers for two big reasons:
- Ease of integration
- Consistent standards


## Separation of concerns

---

In order to talk about APIs, we need first to introduce the _separation of concerns_. In computer science, _separation of concerns_ (SoC) is a design principle for separating a computer program into distinct sections, such that each section addresses a separate concern. A concern is a set of informations that affects the code of a computer program. 

In particular, when building a web application, it's best practice to separate the website logic from data models. This not only allows for cleaner code, but is an easier way to manipulate our layouts and interactions. Separation of concerns becomes ever more important when working with outside data.

<img src="assets/MVC-Process.png" style="width: 200px;"> 

> _MVC: Model View Controller, is a famous SoC paradigm in programming._ 

API calls are really a fancy term for making _HTTP requests_ (in the context of web APIs) to a server and sending/receiving structured data from that endpoint (URL). We are still communicating with URLs, however instead of receiving markup, like we do with HTML pages, we receive data.

[Representational state transfer (REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) is the most common architecture style for passing information to and from these API endpoints, because it allows us to hit a server with an HTTP request (which can be written in almost any language) and returns data in a standard format, like JSON.

Before we start consuming these services however, it's important to understand the fundamentals of the underlying communication layer: **HTTP**.


## HTTP

---

HTTP is a protocol - a system of rules - that determines how web pages (see:'hypertext') get sent (see:'transferred') from one place to another. Among other things, it defines the format of the messages passed between HTTP clients and HTTP servers.

Since the web is a service, it works through a combination of clients, which _make_ requests and servers, which _receive_ requests.


### The HTTP client

HTTP Clients make or generate HTTP Requests. Some types of clients are:

* Browsers - Chrome, Firefox and Safari.
* Command Line programs - [curl](http://curl.haxx.se/docs/) and [wget](http://www.gnu.org/software/wget/manual/wget.html).
* Application code -  Python Requets, Scrapy, Mechanize

HTTP Clients respond to HTTP Responses from a Web Server. They process the data being returned from a Web Server, aka HTTP Server.

### HTTP and web servers

All _Web Servers_ receive _HTTP Requests_ and generate _HTTP Responses_. Often Web Servers are just the middleman, passing HTTP Request and Responses between the client and web application. 

<a id='web-app'></a>
## Web applications

---

Web applications are programs that run on a web server, process the HTTP requests that the server receives, and generate HTTP Responses.

![HTTP Request and Response](assets/request-response.png)

Lost? Here's the play-by-play.

1. A client sends a HTTP Request to a HTTP Server running on a remote machine.  
  * The _hostname_ given in the URL, indicates which server will receive the request.  
2. The HTTP server processes the HTTP Request. This may entail passing the request to some Web Application, which creates a HTTP Response.
3. The response gets sent back to the client.
4. The client processes the response.

How does the server know what the request is asking for? This is specified by the URL, a special kind of path that specifies where a resource can be found on the web.

![URL](./assets/http1-url-structure.png)

> **Check:** can anyone explain what a client and a server are?

## Returning to our Darksky example, let's break down the anatomy of this url:
https://api.darksky.net/forecast/YourKeyHere/40.7808,-73.9772,1502150400?exclude=minutely

### Check: Where does the web application live?

Client, server, the cloud, mobile device, in your car, or on your bluetooth connected toaster with LCD display?

## HTTP Request

---

The first word in the request line, _GET_, is the **HTTP Request's Method**.

![HTTP Request](./assets/http_request.jpeg "HTTP Request")

<a id='request-methods'></a>
### HTTP Request methods:

* **`GET`** => Retrieve a resource.  
* **`POST`** => Create a resource.  
* **`PATCH`** (_or **`PUT`**, but **`PATCH`** is recommended_) => Update an existing resource.  
* **`DELETE`** => Delete a resource.  
* **`HEAD`** => Retrieve the headers for a resource.

Of these, **`GET`** and **`POST`** are the most widely used.

### HTTP Request structure:

```
[http request method] [URL] [http version]  
[list of headers]

[request body]
```

*Notice, that the Request Header is separated from the Request Body by a new line.*

In [5]:
#HTTP Request Method Example In Code: (No Body)
r = requests.get("http://vermonster.com")
print(r.request.headers, r.request.body)

({'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'User-Agent': 'python-requests/2.14.2'}, None)


## HTTP Response

---

![HTTP Response](./assets/http_response.jpeg "HTTP Response")

When a client sends a request, the server sends back a response; the standard format for this response is:

```
[http version] [status] [reason]  
[list of headers]

[response body] # typically HTML, JSON, ...  
```

### Response types overview

> Check these out when you have time -- at least be aware that there is an expected pattern to these codes:

**[Status Codes](http://en.wikipedia.org/wiki/List_of_HTTP_status_codes)** have standard meanings; here are a few.

|Code|Reason|
|:---|:-----|
|200| OK
|301| Moved Permanently
|302| Moved Temporarily
|307| Temporary Redirect
|400| Bad Request
|403| Forbidden
|404| Not Found
|500| Internal Server Error

In [30]:
# Response
r, r.headers

(<Response [200]>,
 {'Content-Length': '14902', 'Content-Encoding': 'gzip', 'x-amz-id-2': 'CgC6eUKJ8oeX7eJFC3oTo4MTZEScKsxCr5uAMNnWSZ095aHItkwiWevfzLIOtPXY0+r09s5r8j0=', 'Server': 'AmazonS3', 'Last-Modified': 'Tue, 25 Apr 2017 20:38:40 GMT', 'x-amz-request-id': '88F5189AE2B9AD71', 'Date': 'Sat, 05 Aug 2017 20:25:26 GMT', 'x-amz-meta-content-md5': '967987e254098c0894f2b6fa33c75eac', 'Content-Type': 'text/html; charset=utf-8', 'ETag': '"1d7365e1ac1624885abfbc36c0850107"'})

## Demo: HTTP

---

Lets explore HTTP resources. We'll start by looking at HTTP requests and responses using the Chrome Inspector.

![HTTP Request and Response](./assets/http_request_response.jpeg "HTTP Request and Response")

* In Chrome, open up Chrome Inspector (*command + option + 'i', or ctrl + click and select 'inspect element'*).
* Select the Network tab. It should look something like this:

![Chrome Inspector](./assets/chrome_inspector.png)

* Next, go to the URL https://generalassemb.ly/

You should be able to see a few HTTP Requests and Responses in the Network tab; for each request you'll see a **Path**, **Method**, **Status**, **Type**, and **Size**, along with info about how long it took to get each of these resources.
  * Most of this information comes from the HTTP Request and Response.*
  * Some HTTP requests are for CSS, JavaScript and images that are referenced by the HTML.
  * Select `generalassemb.ly` in the Path column on the far left.
  * Select the Headers tab. **Headers** are meta-data properties of an HTTP request or response, separate from the body of the message.

## Independent practice: HTTP

---


### With a partner, go to your favorite website(s) (safe for work), inspect the protocol from the Chrome network inspection tool, identify:

- Requests sent by your client
- Response sent by the server

#### Research and Explain:
- cache-control
- age
- content-encoding
- expires
- "GET" and "POST" requests
- query string parameters

## JSON

---

JSON is short for _JavaScript Object Notation_, and is a way to store information in an organized, easy-to-access manner. In a nutshell, it gives us a human-readable collection of data that we can access in a really logical manner.

**JSON is built on two structures:**
* A collection of name/value pairs. In various languages, this is realized as an object, record, structure, dictionary, hash table, keyed list, or associative array.
* An ordered list of values. In most languages, this is realized as an array, vector, list, or sequence.

These are universal data structures. Virtually all modern programming languages support them in one form or another. It makes sense that a data format that is interchangeable with programming languages also be based on these structures.

### JSON objects

An object is an unordered set of name/value pairs, like python dictionaries. An object begins with `{` (left brace) and ends with `}` (right brace). Each name is followed by `:` (colon) and the name/value pairs are separated by `,` (comma).

The syntax is as follows:

```
{ string : value, .......}
```
like:
```
{"count": 1, ...}
```
_Seems an awful lot like a python dictionary._

## Independent practice: validating JSON 

---

JSON is very simple to use if correctly structured. One of the resources to validate JSON and check if the syntax is correct is [JSON Viewer](http://codebeautify.org/jsonviewer).

For this exercise, copy the [JSON data from the code folder](./code/test.json) and insert it in the web app above, click "Validate".

If you see "Valid JSON", click "Beautify" and you will see a more readable way of JSON. If you do not see the message "Valid JSON", it means that there is a syntax error.

In our case, there are some errors in the JSon, so correct all errors in the code!

## Independent Practice - Solution:

[JSON Solution can be found in the code folder](./code/testJsonSolution.json)

## Guided practice: Pulling data from APIs

---

Recall that APIs are methods and data formats to tell people how to "talk" to a system. We will walk through a couple of examples.

### Example 1: movies

The internet movie database is a large collection of data about movies. It can be browsed at the address: http://www.imdb.com/.

What if we wanted to programatically access the data in the database? Unless we are employees of IMDB.com, we probably don't have direct access to their internal database, so we cannot perform SQL queries on their data.

We could use scraping to retrieve data from the web page, and in some cases we will have to do exactly that.

> *Note: check the "Terms of Service" before you scrape a website, you could be infringing their terms.*

In other cases, the website offers a way to programatically access data from their database. That's an API.

In the case of movies, this is offered by http://www.omdbapi.com/

_Unfortunately, OMDB is no longer a free to use API.  Instead, we'll practice interacting with the Star Wars API (SWAPI)_


**Let's try for example to retrieve the data about the "Obi-Wan Kenobi".**

Referencing the [SWAPI Documentation](http://swapi.co/documentation), lets create a query to search for characters with "obi" in their name.

**In a browser, paste:**

    https://swapi.co/api/people/?format=json&search=obi


{
    "count": 1,   
    "next": null,   
    "previous": null,   
    "results": [  
        {  
            "name": "Obi-Wan Kenobi",   
            "height": "182",   
            "mass": "77",   
            "hair_color": "auburn, white",   
            "skin_color": "fair",   
            "eye_color": "blue-gray",   
            "birth_year": "57BBY",   
            "gender": "male",   
            "homeworld": "http://swapi.co/api/planets/20/",   
            "films": [    
                "http://swapi.co/api/films/2/",   
                "http://swapi.co/api/films/5/",   
                "http://swapi.co/api/films/4/",   
                "http://swapi.co/api/films/6/",   
                "http://swapi.co/api/films/3/",   
                "http://swapi.co/api/films/1/"  
            ],   
            "species": ["http://swapi.co/api/species/1/"],     
            "vehicles": ["http://swapi.co/api/vehicles/38/"],   
            "starships": [  
                "http://swapi.co/api/starships/48/",     
                "http://swapi.co/api/starships/59/",   
                "http://swapi.co/api/starships/64/",   
                "http://swapi.co/api/starships/65/",   
                "http://swapi.co/api/starships/74/"  
            ],   
            "created": "2014-12-10T16:16:29.192000Z",     
            "edited": "2014-12-20T21:17:50.325000Z",     
            "url": "http://swapi.co/api/people/10/"    
        }  
    ]  
}  

###  What Just Happened?

We requested a url, which responded with JSON.

SWAPI has a GUI based response as well which is the default.

https://swapi.co/api/people/?search=obi

Also a Wookie flavored one.

https://swapi.co/api/people/?format=wookiee&search=obi

### Try submitting a couple more queries to familiarize with the API.

- You can also query an API from the command line using the app `curl`. Try typing:

    `curl http://swapi.co/api/people/13/?format=json`


```javascript   
{"name":"Chewbacca",
 "height":"228",
 "mass":"112",
 "hair_color":"brown",
 "skin_color":"unknown",
 "eye_color":"blue",
 "birth_year":"200BBY",
 "gender":"male",
 "homeworld":"http://swapi.co/api/planets/14/",
 "films": ["http://swapi.co/api/films/2/","http://swapi.co/api/films/6/","http://swapi.co/api/films/3/","http://swapi.co/api/films/1/","http://swapi.co/api/films/7/"],
 "species":["http://swapi.co/api/species/3/"],
 "vehicles":["http://swapi.co/api/vehicles/19/"],
 "starships":["http://swapi.co/api/starships/10/","http://swapi.co/api/starships/22/"],
 "created":"2014-12-10T16:42:45.066000Z",
 "edited":"2014-12-20T21:17:50.332000Z",
 "url":"http://swapi.co/api/people/13/"}
```

In [1]:
# Request example for the IMDB example
import pandas as pd
import requests
result = requests.get("http://swapi.co/api/people/10/")
df = pd.DataFrame([result.json()])
df

,birth_year,created,edited,eye_color,films,gender,hair_color,height,homeworld,mass,name,skin_color,species,starships,url,vehicles
0,57BBY,2014-12-10T16:16:29.192000Z,2014-12-20T21:17:50.325000Z,blue-gray,"[http://swapi.co/api/films/2/, http://swapi.co...",male,"auburn, white",182,http://swapi.co/api/planets/20/,77,Obi-Wan Kenobi,fair,[http://swapi.co/api/species/1/],"[http://swapi.co/api/starships/48/, http://swa...",http://swapi.co/api/people/10/,[http://swapi.co/api/vehicles/38/]


<a id='ex2-geocode'></a>
### Example 2: Google Geocode API

Google offers a freely accessible API to query their GEO databases.  One of the many features Google Maps API provides is a way to get longitude and latitude coordinates from addresses.

**Try pasting the following line in your browser:**

    https://maps.googleapis.com/maps/api/geocode/json?address=225+Bush+St+San+Francisco+CA

In [7]:
# Request the resource from google maps
result = requests.get("https://maps.googleapis.com/maps/api/geocode/json?address=225+Bush+St+San+Francisco+CA")
google_result = result.json()

# Loop through results and display lat, lon values for reverse geocode
for item in google_result['results']:
    print item['geometry']['location']

{u'lat': 37.790841, u'lng': -122.4012802}


## oAuth

---

oAuth is simply a secure authorization protocol that deals with the authorization of third party application to access the user data without exposing their password. (e.g., Login with facebook, gPlus, twitter in many websites.) all work under this protocol.

Basically there are three parties involved: **oAuth Provider, oAuth Client and Owner**.

- oAuth Client (Application Which wants to access your credential)
- oAuth Provider (eg. facebook, twitter...)
- Owner (the one with facebook,twitter.. account )

Many APIs are free to access. You first need to register as a developer and obtain an authorization key. In most cases, this is also accompanied by a temporary token that needs to be renewed after some time. This is a way to prevent abuse on the server's resources.

You can read more about it here: http://oauth.net/2/


### OAuth Example:  client work with instagram

![](assets/oAuth1.png)

## Independent Practice: Python APIs

---

**Form pairs and do the following:**  
Go to http://www.pythonforbeginners.com/api/list-of-python-apis  or https://github.com/realpython/list-of-python-api-wrappers
  
- Choose 1 API: what data is available with your choosen API?
- Install python module (if available for API), try to extract data.
- Discuss: How could you leverage that api? How could you use the data?

## Closing questions

---

![](assets/closing.png)

### - How does this contrast to scraping?

### - How would you explain API's to someone who didn't know anything about them?

### - Any new ideas for capstone data?